In [1]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import nibabel as nib
import matplotlib.pyplot as plt
from dataset import CarotidDataset
from unet import UNet

from torchvision import transforms

In [2]:
data = CarotidDataset()

epochs = 2
BATCH_SIZE = 4
net = UNet(in_channels=3, n_classes=1, padding=True, up_mode='upsample')
device = torch.device("mps")
#device = torch.device("cpu")
trainloader = torch.utils.data.DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
step_count = 0
loss = nn.CrossEntropyLoss()

transforms = torch.nn.Sequential(
    transforms.CenterCrop((736, 704))
)

In [3]:
net.train()
net = net.to(device)
train_running_loss = 0.0
train_running_corrects = 0

In [4]:
inputs, labels = next(iter(trainloader))
inputs = inputs.to(device)
labels = labels.to(device)
optimizer.zero_grad()
preds = net(inputs)
preds = F.pad(preds, (3,2,7,6))

In [5]:
preds.shape

torch.Size([4, 1, 749, 709])

In [11]:
preds.sum(dim=[1,2,3])

tensor([175156.5469, 135024.0156, 111581.8047,  73949.6016], device='mps:0',
       grad_fn=<SumBackward1>)

In [12]:
labels.sum(dim=[1,2,3])

tensor([3629160, 4136355, 3182910, 3229830], device='mps:0')

In [16]:
diceLabel = labels.sum(dim=[1,2,3])
dicePrediction = preds.sum(dim=[1,2,3])
diceCorrect = (preds * labels).sum(dim=[1,2,3])
epsilon = 1

In [17]:
diceRatio = (2. * diceCorrect + epsilon)/(dicePrediction + diceLabel + epsilon)

In [18]:
diceRatio

tensor([0.6328, 0.2822, 0.5022, 0.1761], device='mps:0',
       grad_fn=<DivBackward0>)

In [19]:
1 - diceRatio

tensor([0.3672, 0.7178, 0.4978, 0.8239], device='mps:0',
       grad_fn=<RsubBackward1>)

In [6]:
preds.view(-1).shape

torch.Size([2124164])

In [7]:
labels.shape

torch.Size([4, 1, 749, 709])

In [ ]:
train_loss = dice_loss(preds,labels)

In [34]:
class DiceLossV1(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, preds, labels, epsilon=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #preds = F.sigmoid(preds)   
            
        diceLabel = labels.sum(dim=[1,2,3])
        dicePrediction = preds.sum(dim=[1,2,3])
        diceCorrect = (preds * labels).sum(dim=[1,2,3])

        diceRatio = (2. * diceCorrect + epsilon)/(dicePrediction + diceLabel + epsilon)       
        return (1 - diceRatio).mean()
dice_loss = DiceLoss()

In [35]:
train_loss = dice_loss(preds, labels)

In [36]:
train_loss

tensor(0.6017, device='mps:0', grad_fn=<MeanBackward0>)